<center><h1>Jupyter notebook corpus</h1></center>
<center><h3>Агроскин Александр</h3></center>

Доступные файлы берем из файла `addresses.txt` в папке `notebook`.

In [1]:
with open("notebooks/addresses.txt", 'r') as addr:
    addresses = addr.read().splitlines()

Для скачивания файлов, работает только если система поддерживает команду `wget`. В папке предзагружены пять штук.

In [2]:
# Не работает на Windows, для загрузки ноутбуков в первый раз
# import os

# for addr in addresses:
#     os.system('wget -P notebooks https://github-notebooks-update1.s3-eu-west-1.amazonaws.com/' + addr)

Выбираю ноутбук и читаю его в формате JSON. Получаю словарь.

In [3]:
import json

working_addr = addresses[0] # Выбираю какой notebook анализируем (от 0 до 4)
with open("notebooks/" + working_addr, 'r') as nb:
    data = json.load(nb)

Версия питона находится в `metadata`, версия Юпитера состоит из `nbformat` и `nbformat_minor`.

In [4]:
name = data["metadata"]["language_info"]["name"]
version = data["metadata"]["language_info"]["version"]
print(f"Language and version:\t{name} {version}")
nbversion = str(data["nbformat"]) + "." + str(data["nbformat_minor"])
print(f"Jupyter format version:\t{nbversion}")

Language and version:	python 3.5.4
Jupyter format version:	4.1


Использую `pandas` для создания таблицы клеток из словаря. Так как id нету у ноутбуков, созданных на версиях `nbformat`, меньших 5.1.0, у подавляющего большинства объектов датасета нельзя сделать колонку `cell_id`. Как внутренний id я пронумеровываю ячейки ноутбука вручную.

In [5]:
import pandas as pd

data_table = pd.DataFrame(data["cells"])
data_table.index.name = "cell_id"
cell_table = data_table.drop(['metadata', 'execution_count', 'outputs'], axis=1)
cell_table.head()

,cell_type,source
cell_id,,
0,markdown,"[# CNTK 207: Sampled Softmax\n, \n, For classi..."
1,markdown,[**Select the notebook runtime environment dev...
2,code,[from __future__ import print_function # Use a...
3,markdown,"[## Basic concept\n, \n, The softmax function ..."
4,code,[# Creates a subgraph computing cross-entropy ...


Сделаю отдельную таблицу для ячеек с кодом, где по `cell_id` ячейки можно получить количество вызовов функций в ней:

In [6]:
code_table = data_table.drop(['metadata', 'outputs'], axis=1)
code_table = code_table[code_table.cell_type == "code"].drop('cell_type', axis=1)
code_table.head()

,source,execution_count
cell_id,,
2,[from __future__ import print_function # Use a...,1.0
4,[# Creates a subgraph computing cross-entropy ...,2.0
6,[# Creates subgraph computing cross-entropy wi...,3.0
8,"[import numpy as np\n, from math import log, e...",4.0
10,"[# We want to lot the data \n, import matplotl...",5.0


Для определения языка текста воспользуюсь библиотекой [Compact Language Detector v3](https://github.com/google/cld3) с [байндингами для питона](https://pypi.org/project/gcld3/). Согласно [этой статье](https://arxiv.org/pdf/1801.07779.pdf), CLD2 (предшественник CLD3) дает очень хорошее качество определения, и при этом работает на порядок быстрее похожих библиотек.

Создам таблицу, в которой по `cell_id` маркдаун ячейки можно узнать язык текста:

In [7]:
# Для установки библиотеки в первый раз
# !pip install gcld3
# Для установки словаря-списка языков (надеюсь, что я не нарушил лицензию)
# !wget https://raw.githubusercontent.com/unicode-cldr/cldr-localenames-modern/master/main/en/languages.json

In [8]:
import gcld3, re


def detect_lang(strarr, identifier, lang_map):
    # Удаляю из строки знаки препинания, может помочь с определением языка 
    string = ' '.join([re.sub(r'[^\w\s]', ' ', s) for s in strarr]).replace('\n', '').lower()
    res = identifier.FindLanguage(string)
    return lang_map[res.language], res.is_reliable

lang_table = data_table.drop(['metadata', 'outputs'], axis=1)
lang_table = lang_table[lang_table.cell_type == "markdown"].drop(['cell_type', 'execution_count'], axis=1)
with open("languages.json", 'r') as lang: # Загружаю словарь для полных имен
    lang_map = json.load(lang)['main']['en']['localeDisplayNames']['languages']
identifier = gcld3.NNetLanguageIdentifier(0, 2000)  # Первых 2000 байт должно хватить для определения языка
lang_table[['language', 'is_reliable']] = lang_table['source'].apply(lambda src: detect_lang(src, identifier, lang_map)).tolist()
lang_table.head(10)

,source,language,is_reliable
cell_id,,,
0,"[# CNTK 207: Sampled Softmax\n, \n, For classi...",English,True
1,[**Select the notebook runtime environment dev...,English,True
3,"[## Basic concept\n, \n, The softmax function ...",English,True
5,[To give a better idea of what the inputs and ...,English,True
7,[As you can see the main differences to the ap...,English,True
9,[In the above code we use two different method...,English,True
11,[In the example above we compare uniform sampl...,English,True
12,"[## What speedups to expect?\n, \n, The speed ...",English,True


Получили таблицу, где в колонке `language` находится язык текста в ячейке, а в колонке `is_reliable` информация о том, уверена модель в своем ответе или нет.

Сохраню все таблицы в отдельную папку:

In [9]:
from pathlib import Path

Path('tables/' + working_addr[:-6]).mkdir(parents=True, exist_ok=True)
cell_table.to_csv('tables/' + working_addr[:-6] + '/cells.csv')
code_table.to_csv('tables/' + working_addr[:-6] + '/code.csv')
lang_table.to_csv('tables/' + working_addr[:-6] + '/markdown.csv')